In [1]:
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
import pytorch_lightning as pl
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/anujay/miniconda3/envs/asap/lib/python3.8/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv("airtel.csv")
df['Date'] = range(len(df))
df['dummy'] = 0 # Dummy group added for bug in Pytorch-Forecasting when these is no group.

In [3]:
print(df.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'dummy'], dtype='object')


In [4]:
dataset = TimeSeriesDataSet(
    df,
    time_idx="Date",
    target="Close",
    group_ids=['dummy'],
    min_encoder_length=7,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=30,
    min_prediction_length=1,
    max_prediction_length=1,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    variable_groups={},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["Date"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["Volume"],
    target_normalizer=None,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [5]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time
validation = TimeSeriesDataSet.from_dataset(dataset, df, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 10  # set this between 32 to 128
train_dataloader = dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=7)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [6]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
MAE()(baseline_predictions.output, baseline_predictions.y)

/home/anujay/miniconda3/envs/asap/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/anujay/miniconda3/envs/asap/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/home/anujay/miniconda3/envs/asap/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or

tensor(9.5000)